In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [4]:
a = df.isnull()
sum = a.sum()
print(sum)


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [5]:
mean_val = df['Age'].mean()
print(mean_val)

29.69911764705882


In [6]:
df['Age'].fillna(value=abs(mean_val), inplace=True)

In [7]:
df['Embarked'].fillna(value="NA",inplace=True)

In [8]:
b = df.isnull()
an = b.sum()
print(an)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [9]:
df.drop('Cabin',axis=1,inplace=True)

In [10]:
df['Sex'] = np.where(df['Sex'] == "female",0,1)

In [11]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,S


In [12]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

In [13]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C,2
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,S,1


In [14]:
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

In [15]:
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 40, 60, 100], labels=[0, 1, 2, 3, 4])

In [16]:
df['FarePerPerson'] = df['Fare'] / df['FamilySize']

In [17]:
df.drop(['Name','Ticket'],axis=1,inplace=True)

In [18]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder

# Encode categorical features like 'Sex' and 'Embarked'
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])  # Convert 'Sex' into numeric (0,1)
df['Embarked'] = df['Embarked'].fillna('S')  # Fill missing 'Embarked'
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])  # Convert 'Embarked' to numeric

# Alternatively, use one-hot encoding for 'Embarked' if needed:
# df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

# Target variable 'Survived'
X = df.drop(['Survived'], axis=1)  # Features
y = df['Survived']  # Target

# Perform Chi-Square test for univariate selection
k_best = SelectKBest(score_func=chi2, k=10)
X_k_best = k_best.fit_transform(X.fillna(0), y)

# Display selected feature scores
feature_scores = pd.DataFrame(k_best.scores_, index=X.columns, columns=['Score'])
print(feature_scores.nlargest(10, 'Score'))




                     Score
Fare           4518.319091
FarePerPerson  2818.936692
Sex              92.702447
Pclass           30.873699
Age              24.687926
Embarked         15.844071
IsAlone          14.640793
Parch            10.097499
PassengerId       3.312934
AgeGroup          3.170250


In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Logistic regression model
model = LogisticRegression(max_iter=1000)

# RFE to select top 10 features
rfe = RFE(model, n_features_to_select=10)
fit = rfe.fit(X.fillna(0), y)

# Get the selected features
selected_features_rfe = pd.DataFrame({
    'Feature': X.columns,
    'Selected': rfe.support_,
    'Ranking': rfe.ranking_
})
print(selected_features_rfe[selected_features_rfe['Selected'] == True])


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


       Feature  Selected  Ranking
1       Pclass      True        1
2          Sex      True        1
3          Age      True        1
4        SibSp      True        1
5        Parch      True        1
6         Fare      True        1
7     Embarked      True        1
8   FamilySize      True        1
9      IsAlone      True        1
10    AgeGroup      True        1


In [20]:
from sklearn.ensemble import RandomForestClassifier

# Random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X.fillna(0), y)

# Get feature importances
importances = rf_model.feature_importances_

# Create a DataFrame for feature importance
feature_importances_rf = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

# Display top 10 features
print(feature_importances_rf.nlargest(10, 'Importance'))


          Feature  Importance
2             Sex    0.236828
0     PassengerId    0.158021
3             Age    0.130362
6            Fare    0.128362
11  FarePerPerson    0.127238
1          Pclass    0.065088
8      FamilySize    0.039416
10       AgeGroup    0.039295
7        Embarked    0.026099
4           SibSp    0.022080


In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming 'df' is your DataFrame with features and 'Survived' as target

# Drop the target variable 'Survived' and irrelevant columns if any
X = df.drop(['Survived'], axis=1)

# Encode categorical features like 'Sex' and 'Embarked'
X['Sex'] = LabelEncoder().fit_transform(X['Sex'])  # Convert 'Sex' into numeric (0,1)
X['Embarked'] = X['Embarked'].fillna('S')  # Fill missing 'Embarked'
X['Embarked'] = LabelEncoder().fit_transform(X['Embarked'])  # Convert 'Embarked' to numeric

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.fillna(0))  # Handle missing data

# Apply PCA to reduce to 2 components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the 2 principal components
pca_df = pd.DataFrame(data=X_pca, columns=['Principal Component 1', 'Principal Component 2'])

# If you want to add the target variable back for visualization or analysis:
pca_df['Survived'] = df['Survived'].values

# Display the first few rows
print(pca_df.head())


   Principal Component 1  Principal Component 2  Survived
0               0.634687              -1.186540         0
1               0.608072               2.381831         1
2              -0.634173              -1.006217         1
3               0.673659               1.379393         1
4              -1.196851              -1.160216         0
